# Velocity Data Download Script for Validation
#### Working Script to pull large velocity datasets and later compare to ICESAT2-derived velocities

ICESat-2 hackweek  
June 15, 2020  
Lynn Kaluzienski

# Import necessary modules

In [55]:
import os
import requests
import zipfile

In [56]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import scipy, sys, os, pyproj, glob
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
#! cd ..; [ -d pointCollection ] || git clone https://www.github.com/smithB/pointCollection.git
#sys.path.append(os.path.join(os.getcwd(), '..'))

!python3 -m pip install --user git+https://github.com/tsutterley/pointCollection.git@pip
import pointCollection as pc
#stuck here, not sure how to import point collection 


  Cloning https://github.com/tsutterley/pointCollection.git (to revision pip) to /tmp/pip-req-build-uz4b1u0m
  Running command git clone -q https://github.com/tsutterley/pointCollection.git /tmp/pip-req-build-uz4b1u0m
  Running command git checkout -b pip --track origin/pip
  Switched to a new branch 'pip'
  Branch 'pip' set up to track remote branch 'pip' from 'origin'.
  Created wheel for pointCollection: filename=pointCollection-1.0.0.0-py3-none-any.whl size=82220 sha256=116804956c819e0d40adbb8a6f78e37547b5581510d719f25157c8e2874a0636
  Stored in directory: /tmp/pip-ephem-wheel-cache-g_srn87_/wheels/29/6e/f2/1435b756513140ff2fc1dc41847ebca181fd3285efc91b3d86
Successfully built pointCollection


In [58]:
#Magic function to enable interactive plotting (zoom/pan) in Jupyter notebook
#If running locally, this would be `%matplotlib notebook`, but since we're using Juptyerlab, we use widget
%matplotlib widget
#%matplotlib inline

In [61]:
'''
##Section where I played around with loading in .mat file, quickly abandoned
#load in data from .mat file
#Data originally in netcdf format, created subset in matlab for FIS region and saved as .mat file. 
import scipy.io as spio
data_root='/srv/surface_velocity/contributors/lynn_kaluzienski/Data_test'
#('Measures2.mat',squeeze_me=True)
Measures2_Files= spio.loadmat('./Data_test/Measures2.mat')
x=Measures2_Files['posx']
y=Measures2_Files['posy']
vx=Measures2_Files['vx_FIS']
vy=Measures2_Files['vy_FIS']
vel=Measures2_Files['vel_FIS']

#xy=np.array(pyproj.Proj(3031)(x, y))
plt.figure()
ax.imshow(vel, extent=[0, 1, 0, 1])
'''

"\n#load in data from .mat file\n#Data originally in netcdf format, created subset in matlab for FIS region and saved as .mat file. \nimport scipy.io as spio\ndata_root='/srv/surface_velocity/contributors/lynn_kaluzienski/Data_test'\n#('Measures2.mat',squeeze_me=True)\nMeasures2_Files= spio.loadmat('./Data_test/Measures2.mat')\nx=Measures2_Files['posx']\ny=Measures2_Files['posy']\nvx=Measures2_Files['vx_FIS']\nvy=Measures2_Files['vy_FIS']\nvel=Measures2_Files['vel_FIS']\n\n#xy=np.array(pyproj.Proj(3031)(x, y))\nplt.figure()\nax.imshow(vel, extent=[0, 1, 0, 1])\n"

In [62]:
#From Ben Smith's code loading in .tif file, running into issues likely with directories
data_root='/surface_velocity/contributors/lynn_kaluzienski'
#spatial_extent = np.array([-102, -76, -98, -74.5])
spatial_extent = np.array([-65, -86, -55, -81])
lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
print(lat)
print(lon)
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
#Originally tried to load data from a local directory, should change to shared directory
Measures_vel=pc.grid.data().from_geotif(os.path.join(data_root,'Data_test','Measures2_FIS.tif'), bounds=[XR, YR])



# show the velocity map:
plt.figure()
Measures_vel.show(cmap='gray', clim=[14000, 17000])
plt.plot(xy[0,:], xy[1,:])
plt.title('Measures2 Velocity Map')


[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]


AttributeError: 'NoneType' object has no attribute 'GetGeoTransform'

In [49]:
#Section where I started trying rasterio
import rasterio
import hvplot
import hvplot.xarray
import xarray as xr
Measures2_file = os.path.join(wd,'/home/jovyan/surface_velocity/contributors/lynn_kaluzienski/Data_test/Measures2_FIS.tif')

src = rasterio.open(Measures2_file)

da = xr.open_rasterio(src)
subplot_width  = int(src.shape[1] /9)
subplot_height = int(src.shape[0] /10)
da.sel(band=1).hvplot.image(rasterize=True,width=subplot_width,
                            height=subplot_height,
                            cmap='gray')


NameError: name 'wd' is not defined